In [2]:
import openturns as ot
import numpy as np

j = 28
A = j**3 / (32 - j)


X = ot.Uniform(-A, A)
Y = ot.Uniform(-A, A)
T = ot.Uniform(-np.pi/j, np.pi/j)
distribution = ot.ComposedDistribution([X, Y, T])


model = ot.SymbolicFunction(
    ["X", "Y", "T"],
    ["cos(T)*X + sin(T)*Y"]
)

In [3]:
# Question 1
N = 200
input_sample = distribution.getSample(N)
output_sample = model(input_sample)

print("Q1 done")

Q1 done


In [4]:
# Question 2 Spearman
corr = ot.CorrelationAnalysis(input_sample, output_sample)
spearman = corr.computeSpearmanCorrelation()

print("Q2 — Spearman:", spearman)


Q2 — Spearman: [0.997025,-0.0933338,-0.0354384]


In [5]:
# Question 3 — Kriging
basis = ot.ConstantBasisFactory(3).build()
cov = ot.SquaredExponential([1.0, 1.0, 0.3])

kriging_algo = ot.KrigingAlgorithm(
    input_sample,
    output_sample,
    cov,
    basis
)
kriging_algo.run()
kriging_model = kriging_algo.getResult().getMetaModel()

print("Q3 done")

Q3 done


In [ ]:
# Question 4 — Sobol (Saltelli + méta-modèle)
N_sobol = 1000

experiment = ot.SaltelliExperiment(distribution, N_sobol)
X_sobol = experiment.generate()

Y_sobol = kriging_model(X_sobol)

sobol_q4 = ot.SobolIndicesAlgorithm(X_sobol, Y_sobol, N_sobol)

S1 = sobol_q4.getFirstOrderIndices()
ST = sobol_q4.getTotalOrderIndices()

print("Q4 — Sobol S1:", S1)
print("Q4 — Sobol ST:", ST)


In [6]:
# Question 5 — Spearman
for N in [50, 100, 200, 500]:
    input_sample = distribution.getSample(N)
    output_sample = model(input_sample)

    corr = ot.CorrelationAnalysis(input_sample, output_sample)
    spearman = corr.computeSpearmanCorrelation()

    print(f"Q5 — N={N}, Spearman={spearman}")

Q5 — N=50, Spearman=[0.997119,-0.0442257,-0.0655462]
Q5 — N=100, Spearman=[0.996376,0.0657906,0.179262]
Q5 — N=200, Spearman=[0.997178,0.0771379,-0.0533368]
Q5 — N=500, Spearman=[0.99778,-0.0498016,-0.0815113]
